In [1]:

import tensorflow.compat.v1 as tf
import numpy as np
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2022-02-10 16:16:16.690818: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-10 16:16:17.620984: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22345 MB memory:  -> device: 0, name: Graphics Device, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-02-10 16:16:17.622143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22345 MB memory:  -> device: 1, name: Graphics Device, pci bus id: 0000:65:00.0, compute capability: 8.6


In [2]:
!which python

/home/gmpark/anaconda3/envs/ed/bin/python


In [3]:
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))

Number of devices: 2


# export 생략하고 inference
#### 사전 export된 모델을 각 gpu로 읽어와서 실행 가능하기 때문에, export과정은 gpu mapping 필요 없음 

In [4]:
#default flag setting

flag = {}

flag['model_name'] = 'efficientdet-d0'
flag['logdir'] = '/tmp/deff/'
flag['runmode'] = 'dry'
flag['trace_filename'] = None

flag['threads'] = 0
flag['bm_runs'] = 10
flag['tensorrt'] = None
flag['delete_logdir'] =True
flag['freeze'] = False
flag['use_xla'] =False
flag['batch_size'] = 1

flag['ckpt_path'] = None
flag['export_ckpt'] = None

flag['hparams'] = ''
flag['input_image'] = None
flag['output_image_dir'] = None

flag['input_video'] = None
flag['output_video'] = None

flag['line_thickness'] = None
flag['max_boxes_to_draw'] = 100
flag['min_score_thresh'] = 0.4
flag['nms_method'] = 'hard'

flag['saved_model_dir'] = '/tmp/saved_model'
flag['tfile_path'] = None

In [5]:
# export parameter

MODEL = 'efficientdet-d0'  #@param

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz
    !tar zxf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model in {}'.format(ckpt_path))


flag['runmode'] = 'saved_model'
flag['model_name'] = 'efficientdet-d0'
flag['ckpt_path'] = ckpt_path
flag['hparams'] = 'image_size=1920x1280'
flag['saved_model_dir'] = 'savedmodel'

Use model in /home/gmpark/automl/efficientdet/efficientdet-d0


In [6]:
# inference parameter

flag['runmode'] = 'saved_model_infer'
flag['model_name'] = 'efficientdet-d0'
flag['saved_model_dir'] = 'savedmodel'
flag['input_image'] = 'img.png'
flag['output_image_dir'] = 'serve_image_out'
flag['min_score_thresh'] = 0.35
flag['max_boxes_to_draw'] = 200

In [7]:
from p_model_inspect import P_ModelInspector
from PIL import Image

with mirrored_strategy.scope():
  tf.disable_eager_execution()
  inspector = P_ModelInspector(
    model_name=flag['model_name'],
    logdir=flag['logdir'],
    tensorrt=flag['tensorrt'],
    use_xla=flag['use_xla'],
    ckpt_path=flag['ckpt_path'],
    export_ckpt=flag['export_ckpt'],
    saved_model_dir=flag['saved_model_dir'],
    tflite_path=flag['tfile_path'],
    batch_size=flag['batch_size'],
    hparams=flag['hparams'],
    score_thresh=flag['min_score_thresh'],
    max_output_size=flag['max_boxes_to_draw'],
    nms_method=flag['nms_method'])
  
  
  import tensorflow.compat.v1 as tf
  sess_config = tf.ConfigProto()
  sess = tf.Session(config=sess_config)
  signitures={
        'image_files': 'image_files:0',
        'image_arrays': 'image_arrays:0',
        'prediction': 'detections:0',}
  if tf.io.gfile.isdir(flag['saved_model_dir']):
    tf.saved_model.load(sess, ['serve'],
                                 flag['saved_model_dir'])
  


  

run_mdoel_inspect...
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


2022-02-10 16:16:19.387316: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22345 MB memory:  -> device: 0, name: Graphics Device, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-02-10 16:16:19.388156: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22345 MB memory:  -> device: 1, name: Graphics Device, pci bus id: 0000:65:00.0, compute capability: 8.6


INFO:tensorflow:Restoring parameters from savedmodel/variables/variables


In [8]:
from p_model_inspect import P_ModelInspector
from PIL import Image

def parameter_step():
  '''
  driver = inspector.run_model(
      flag['runmode'],
      input_image=flag['input_image'],
      output_image_dir=flag['output_image_dir'],
      input_video=flag['input_video'],
      output_video=flag['output_video'],
      line_thickness=flag['line_thickness'],
      max_boxes_to_draw=flag['max_boxes_to_draw'],
      min_score_thresh=flag['min_score_thresh'],
      nms_method=flag['nms_method'],
      bm_runs=flag['bm_runs'],
      threads=flag['threads'],
      trace_filename=flag['trace_filename'])
    # Serving time batch size should be fixed.
    '''
    
  
  batch_size = flag['batch_size'] or 1
  all_files = list(tf.io.gfile.glob(flag['input_image']))
  print('all_files=', all_files)
  num_batches = (len(all_files) + batch_size - 1) // batch_size

  for i in range(num_batches):
    batch_files = all_files[i * batch_size:(i + 1) * batch_size]
    height, width = (1280, 1920) # 직접입력
    images = [Image.open(f) for f in batch_files]
    if len(set([m.size for m in images])) > 1:
      # Resize only if images in the same batch have different sizes.
      images = [m.resize(height, width) for m in images]
    raw_images = [np.array(m) for m in images]
    size_before_pad = len(raw_images)
    if size_before_pad < batch_size:
      padding_size = batch_size - size_before_pad
      raw_images += [np.zeros_like(raw_images[0])] * padding_size
    
    #serve image
    detections_bs = sess.run(signitures['prediction'],feed_dict={signitures['image_arrays']:raw_images})
    
    for j in range(size_before_pad):
      #img = driver.visualize(raw_images[j], detections_bs[j], max_boxes_to_draw=200,min_score_thresh=0.35)
      
      #visualize_image_prediction(image, prediction,label_map=self.label_map,**kwargs)
      
      boxes = detections_bs[j][:, 1:5]
      classes = detections_bs[j][:, 6].astype(int)
      scores = detections_bs[j][:, 5]
      from tf2 import label_util
      from visualize import vis_utils
      '''
      #visualize_image(raw_images[j],boxes,classes,scores,None,max_boxes_to_draw=200,min_score_thresh=0.35)
      label_map = label_util.get_label_map(None or 'coco')
      category_index = {k: {'id': k, 'name': label_map[k]} for k in label_map}
      img = np.array(raw_images[j])
      vis_utils.visualize_boxes_and_labels_on_image_array(
      img,
      boxes,
      classes,
      scores,
      category_index,
      min_score_thresh=flag['min_score_thresh'],
      max_boxes_to_draw=flag['max_boxes_to_draw'],
      line_thickness=flag['line_thickness'],
      )
      
      
      img_id = str(i * batch_size + j)
      output_image_path = os.path.join(flag['output_image_dir'], img_id + '.jpg')
      Image.fromarray(img).save(output_image_path)
      print('writing file to %s' % output_image_path)
      '''
  print('run')

  return 0

In [16]:
# model export
@tf.function

def distributed_run_step():
    per_replica = mirrored_strategy.run(parameter_step)
    return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM,per_replica,axis=None)
distributed_run_step()


all_files= ['img.png']
run
all_files= ['img.png']
run


<tf.Tensor 'PartitionedCall_2:0' shape=() dtype=int32>

In [ ]:

def parameter_step():
  #parameter setting
  runmode='saved_model_infer'
  model_name=MODEL
  saved_model_dir = 'savedmodel'
  input_image='img.png'
  serve_image_out = 'serve_image_out'
  output_image_dir='serve_image_out'
  image_dir=serve_image_out
  max_boxes_to_draw=100
  min_score_thresh=0.4
  batch_size=1
  min_score_thresh=min_score_thresh
  max_boxes_to_draw=max_boxes_to_draw


  
  driver = inspector.run_model(
    runmode,
    input_image=input_image,
    output_image_dir=output_image_dir,
    input_video=input_video,
    output_video=output_video,
    line_thickness=line_thickness,
    max_boxes_to_draw=max_boxes_to_draw,
    min_score_thresh=min_score_thresh,
    nms_method='hard',
    bm_runs=10,
    threads=0,
    trace_filename=None)
  
    
  return 0

In [ ]:
# model export
@tf.function
def distributed_run_step():
    per_replica = mirrored_strategy.run(parameter_step)
    return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM,per_replica,axis=None)
results = distributed_run_step()
print(results)

In [ ]:
http://hpkim0512.blogspot.com/2017/11/tensorflow-multi-gpu.html